In [ ]:
import os
import torch
import time
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
from tqdm import tqdm

from data.dataset2 import RoadDatasetSegFormer
from loss.loss import bce_dice_loss
from utils.early_stopping import EarlyStopping
from utils.save_load import save_model
from config.optimizer import get_optimizer, get_scheduler
from utils.metrics import compute_metrics

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
batch_size = 4
num_epochs = 50
lr = 1e-4
log_dir = 'logs'
model_path = 'checkpoints/segformer_model_best.pth'
csv_path = 'data/512/split.csv'
image_dir = 'data/512/images'
mask_dir = 'data/512/masks'

In [ ]:
train_dataset = RoadDatasetSegFormer(csv_path, image_dir, mask_dir, split='train', augment=True)
val_dataset = RoadDatasetSegFormer(csv_path, image_dir, mask_dir, split='val', augment=False)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

In [ ]:
from transformers import SegformerForSemanticSegmentation
num_classes = 1

model = SegformerForSemanticSegmentation.from_pretrained(
    'nvidia/segformer-b1-finetuned-ade-512-512',
    ignore_mismatched_sizes=True,
    num_labels=num_classes,
)
optimizer = get_optimizer(model, lr)
early_stopping = EarlyStopping(patience=10, mode='min')
scheduler = get_scheduler(optimizer, mode='min')

Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/segformer-b1-finetuned-ade-512-512 and are newly initialized because the shapes did not match:
- decode_head.classifier.weight: found shape torch.Size([150, 256, 1, 1]) in the checkpoint and torch.Size([1, 256, 1, 1]) in the model instantiated
- decode_head.classifier.bias: found shape torch.Size([150]) in the checkpoint and torch.Size([1]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


SegformerForSemanticSegmentation(
  (segformer): SegformerModel(
    (encoder): SegformerEncoder(
      (patch_embeddings): ModuleList(
        (0): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(3, 64, kernel_size=(7, 7), stride=(4, 4), padding=(3, 3))
          (layer_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        )
        (1): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (layer_norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        )
        (2): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(128, 320, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (layer_norm): LayerNorm((320,), eps=1e-05, elementwise_affine=True)
        )
        (3): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(320, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)

In [12]:
writer = SummaryWriter(log_dir)
log_txt = open(os.path.join(log_dir, 'train_log_seg.txt'), 'w')

In [ ]:
# === Training Loop ===
best_dice = 0.0
for epoch in range(num_epochs):
    print(f"\n🟢 Epoch [{epoch+1}/{num_epochs}]")
    model.train()
    train_loss = 0

    for imgs, masks in tqdm(train_loader, desc='Training'):
        imgs, masks = imgs.to(device), masks.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        logits = outputs.logits
        logits = F.interpolate(logits, size=masks.shape[2:], mode='bilinear', align_corners=False)
        preds = torch.sigmoid(logits)
        loss = bce_dice_loss(preds, masks)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    train_loss /= len(train_loader)

    # === Validation ===
    model.eval()
    val_loss = 0
    all_preds = []
    all_masks = []

    with torch.no_grad():
        for imgs, masks in tqdm(val_loader, desc='Validation'):
            imgs, masks = imgs.to(device), masks.to(device)
            outputs = model(imgs)
            logits = outputs.logits
            logits = F.interpolate(logits, size=masks.shape[2:], mode='bilinear', align_corners=False)
            preds = torch.sigmoid(logits)
            loss = bce_dice_loss(preds, masks)
            val_loss += loss.item()

            all_preds.append(preds.cpu())
            all_masks.append(masks.cpu())

    val_loss /= len(val_loader)
    all_preds = torch.cat(all_preds)
    all_masks = torch.cat(all_masks)

    val_dice, val_iou, val_miou = compute_metrics(all_preds, all_masks, threshold=0.5)

    print(f"🔹 Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Dice: {val_dice:.4f} | IoU: {val_iou:.4f} | mIoU: {val_miou:.4f}")
    log_txt.write(f"Epoch {epoch+1}: train_loss={train_loss:.4f}, val_loss={val_loss:.4f}, dice={val_dice:.4f}, iou={val_iou:.4f}, mIoU: {val_miou:.4f}\n")

    # === TensorBoard log ===
    writer.add_scalar('Loss/train', train_loss, epoch)
    writer.add_scalar('Loss/val', val_loss, epoch)
    writer.add_scalar('Dice/val', val_dice, epoch)
    writer.add_scalar('IoU/val', val_iou, epoch)
    writer.add_scalar('mIoU/val', val_miou, epoch)

    # === Scheduler + EarlyStopping ===
    scheduler.step(val_loss)
    early_stopping(val_loss, model)
    if early_stopping.early_stop:
        print("⛔ Early stopping triggered!")
        early_stopping.load_best_weights(model)
        break

    # === Save model if improved ===
    if val_dice > best_dice:
        best_dice = val_dice
        os.makedirs(os.path.dirname(model_path), exist_ok=True)
        save_model(model, model_path, epoch, best_dice)
        print("✅ Model improved. Saved.")

log_txt.close()
writer.close()
print("🏁 Training completed.")



🟢 Epoch [1/50]


Validation: 100%|██████████| 278/278 [00:33<00:00,  8.38it/s]


🔹 Train Loss: 0.7822 | Val Loss: 0.5476 | Dice: 0.5917 | IoU: 0.4407 | mIoU: 0.4407
✅ Model improved. Saved.

🟢 Epoch [2/50]


Validation: 100%|██████████| 278/278 [00:10<00:00, 26.32it/s]


🔹 Train Loss: 0.5503 | Val Loss: 0.5113 | Dice: 0.6166 | IoU: 0.4670 | mIoU: 0.4670
✅ Model improved. Saved.

🟢 Epoch [3/50]


...

🔹 Train Loss: 0.3835 | Val Loss: 0.3963 | Dice: 0.7075 | IoU: 0.5681 | mIoU: 0.5681
✅ Model improved. Saved.

🟢 Epoch [49/50]


Validation: 100%|██████████| 278/278 [00:12<00:00, 22.25it/s]


🔹 Train Loss: 0.3841 | Val Loss: 0.3968 | Dice: 0.7068 | IoU: 0.5672 | mIoU: 0.5672

🟢 Epoch [50/50]


Validation: 100%|██████████| 278/278 [00:12<00:00, 22.05it/s]


🔹 Train Loss: 0.3883 | Val Loss: 0.4035 | Dice: 0.7002 | IoU: 0.5602 | mIoU: 0.5602
🏁 Training completed.
